In [14]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
from pathlib import Path


%matplotlib inline 

In [19]:
data_folder = Path.cwd().parent / "data/data_kmeans.parquet"


In [20]:
dataset = pd.read_parquet(data_folder)

In [ ]:
dataset.info()

In [ ]:
dataset.head()

In [ ]:
grand_i10_variables = dataset.drop(['vin'], axis=1).drop(['year_modelo'], axis=1)
grand_i10_variables.head()

In [ ]:
grand_i10_variables.describe()

### Normalizando los datos

In [ ]:
data_normalizada = (grand_i10_variables-grand_i10_variables.min()) / (grand_i10_variables.max()-grand_i10_variables.min())
data_normalizada.describe()


### Búsqueda de la cantidad optima de clusters
- calculo de la similitud de los individuos dentro de los clusters

In [ ]:
wcss = []

for i in range(1, 15):
    kmeans = KMeans(n_clusters=i, max_iter=300)
    kmeans.fit(data_normalizada)
    wcss.append(kmeans.inertia_)

### Gráfica del codo de Jambú

In [ ]:
plt.plot(range(1,15), wcss)
plt.title("Codo de Jambú")
plt.xlabel("# de Clusters")
plt.ylabel("WCSS")
plt.show()

In [ ]:
clustering = KMeans(n_clusters=5, max_iter=300)
clustering.fit(data_normalizada) ## Aplica el modelo a la base

In [ ]:
print(clustering.inertia_)

### Aquí se agrega la clasificación al archivo original

In [ ]:
dataset["KMeans_Clustering"] = clustering.labels_
dataset.head()

### Visualizando los clusters
- Utilizando PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(data_normalizada)
pca_df = pd.DataFrame(data=pca_results, columns=["Componente_1", "Componente_2"])
pca_df_clusters = pd.concat([pca_df, dataset[["KMeans_Clustering"]]], axis=1)
#pca_df_clusters


In [ ]:
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.set_xlabel("Componente 1")
ax.set_ylabel("Componente 2")
ax.set_title("Componentes principales")

color_theme = np.array(["blue","green", "black","orange", "purple"])
ax.scatter(x=pca_df_clusters.Componente_1, y=pca_df_clusters.Componente_2,
          c=color_theme[pca_df_clusters.KMeans_Clustering], s=5)

plt.show()


In [ ]:
dataset.to_csv("data/i10/output_model_data_grand_i10.csv")